# Climate Data Fetcher

This notebook provides an interactive interface to download climate data from multiple sources:
- Ground station data (via Meteostat)
- ERA5 reanalysis data
- DAYMET data
- PRISM data

## Setup
First, let's import required libraries and set up the interface.

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd

from config import GroundDataConfig, GriddedDataConfig
from src.ground_fetcher import GroundDataFetcher
from src.gridded_fetcher import GriddedDataFetcher
from utils.utils import validate_states, print_summary, compare_datasets

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

## Create Interface Widgets
Now let's create the interactive widgets for data selection.

In [4]:
# Create data type selector
data_type_widget = widgets.RadioButtons(
    options=[('Ground data only', 'ground'),
             ('Gridded data only', 'gridded'),
             ('Both', 'both')],
    description='Data Type:',
    style={'description_width': 'initial'}
)

# Create state selection widgets
state_scope_widget = widgets.RadioButtons(
    options=[('All US States', 'all'),
             ('Select specific states', 'specific')],
    description='States:',
    style={'description_width': 'initial'}
)

# List of US states
us_states = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming'
}

state_selector = widgets.SelectMultiple(
    options=[(f"{code} - {name}", code) for code, name in us_states.items()],
    rows=10,
    description='Select States:',
    disabled=True,
    style={'description_width': 'initial'}
)

# Create year range widgets
start_year = widgets.IntSlider(
    value=1980,
    min=1980,
    max=2024,
    description='Start Year:',
    style={'description_width': 'initial'}
)

end_year = widgets.IntSlider(
    value=2024,
    min=1980,
    max=2024,
    description='End Year:',
    style={'description_width': 'initial'}
)

# Create gridded dataset selection
gridded_datasets = widgets.SelectMultiple(
    options=[('ERA5', 'ERA5'),
             ('DAYMET', 'DAYMET'),
             ('PRISM', 'PRISM')],
    rows=3,
    description='Datasets:',
    disabled=True,
    style={'description_width': 'initial'}
)

# Create output area for results and messages
output_area = widgets.Output()

## Set Up Widget Interactions
Now let's define how the widgets interact with each other.

In [5]:
def on_state_scope_change(change):
    if change['new'] == 'specific':
        state_selector.disabled = False
    else:
        state_selector.disabled = True

state_scope_widget.observe(on_state_scope_change, names='value')

def on_data_type_change(change):
    if change['new'] in ['gridded', 'both']:
        gridded_datasets.disabled = False
    else:
        gridded_datasets.disabled = True

data_type_widget.observe(on_data_type_change, names='value')

# Create download button
download_button = widgets.Button(
    description='Download Data',
    button_style='primary',
    style={'description_width': 'initial'}
)

## Define Download Handler
This function handles the data download process when the button is clicked.

In [6]:
def on_download_button_click(b):
    with output_area:
        clear_output()
        
        # Validate inputs
        if end_year.value < start_year.value:
            print("Error: End year must be greater than or equal to start year")
            return
            
        # Get states
        states = None
        if state_scope_widget.value == 'specific':
            states = list(state_selector.value)
            if not states:
                print("Error: Please select at least one state")
                return
        
        # Process ground data if requested
        results = {}
        if data_type_widget.value in ['ground', 'both']:
            print("Processing ground data...")
            config = GroundDataConfig(
                states=states,
                start_year=start_year.value,
                end_year=end_year.value
            )
            try:
                fetcher = GroundDataFetcher(config)
                ground_data = fetcher.process()
                results['Ground'] = ground_data
                print("Ground data processing complete")
            except Exception as e:
                print(f"Error processing ground data: {e}")
        
        # Process gridded data if requested
        if data_type_widget.value in ['gridded', 'both']:
            print("\nProcessing gridded data...")
            config = GriddedDataConfig(
                start_year=start_year.value,
                end_year=end_year.value
            )
            
            # Enable selected datasets
            selected_datasets = list(gridded_datasets.value)
            for name, dataset in config.datasets.items():
                dataset.enabled = name in selected_datasets
            
            if config.is_valid():
                try:
                    fetcher = GriddedDataFetcher(config)
                    gridded_results = fetcher.process()
                    results.update(gridded_results)
                    print("Gridded data processing complete")
                except Exception as e:
                    print(f"Error processing gridded data: {e}")
            else:
                print("No gridded datasets selected")
        
        # Show results summary
        if results:
            print("\nResults Summary:")
            comparison = compare_datasets(results)
            display(comparison)
        else:
            print("\nNo data was processed successfully")

download_button.on_click(on_download_button_click)

## Display Interface
Finally, let's display the interface for user interaction.

In [7]:
display(HTML("<h2>Climate Data Fetcher</h2>"))
display(widgets.VBox([
    data_type_widget,
    widgets.HBox([start_year, end_year]),
    state_scope_widget,
    state_selector,
    gridded_datasets,
    download_button,
    output_area
]))

In [ ]:
from src.statistical_analyzer import GriddedDataAnalyzer

try:
    print("Starting analysis...")
    analyzer = GriddedDataAnalyzer(data_dir='Data', results_dir='Results')
    analyzer.run_analysis()
    print("\nAnalysis completed successfully!")
    
except Exception as e:
    print(f"\nError during analysis: {str(e)}")
    raise  # Re-raise the exception for debugging if needed

In [ ]:
from src.plot_results import ResultPlotter
"""Main function to run plotting"""
plotter = ResultPlotter()
plotter.run()